In [28]:
from tqdm import tqdm
import json
import networkx as nx
from datetime import datetime

G = nx.Graph()
max_citations = -1

# This json file has all the filtered data provided by OpenAlex API
with open("../data/openalex_cs_papers.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Collecting author and co-authorship features
for work in tqdm(data["results"]):
    authors = []
    cited_by = work.get("cited_by_count", 0)  
    publication_date_str = work.get("publication_date")
    assert publication_date_str is not None
    
    publication_date = datetime.strptime(publication_date_str, '%Y-%m-%d')

    # Adding/updating attributes for each author
    for author_data in work["authorships"]:
        author_id = author_data["author"]["id"]
        affiliation = (
            author_data["institutions"][0]["display_name"]
            if author_data.get("institutions")
            else "Unknown"
        )
        
        authors.append({"id": author_id, "title": work['title']})

        # Custom attributes for author nodes
        if author_id not in G:
            G.add_node(
                author_id,
                affiliated_institution=affiliation,
                citation_count=cited_by
            )
        else:
            G.nodes[author_id]["citation_count"] += cited_by

        max_citations = max(max_citations, G.nodes[author_id]["citation_count"])

    # Adding co-authorship edges
    for i in range(len(authors)):
        for j in range(i + 1, len(authors)):
            id_1, id_2 = authors[i]["id"], authors[j]["id"]
            if G.has_edge(id_1, id_2):
                G[id_1][id_2]["title"].append(authors[i]["title"])
                G[id_1][id_2]["publication_dates"].append(publication_date)
            else:
                G.add_edge(id_1, id_2)
                G[id_1][id_2]["title"] = [authors[i]["title"]]
                G[id_1][id_2]["publication_dates"] = [publication_date]


100%|██████████| 200/200 [00:00<00:00, 713.46it/s]


In [29]:
def get_subgraph_by_degree(graph, N):
    degrees = dict(graph.degree())
    sorted_nodes = sorted(degrees.items(), key=lambda item: item[1])
    nodes_to_remove_count = graph.number_of_nodes() - N
    nodes_to_remove = [node for node, _ in sorted_nodes[:nodes_to_remove_count]]
    nodes_to_retain = [node for node in graph.nodes() if node not in nodes_to_remove]

    return graph.subgraph(nodes_to_retain).copy()

In [30]:
G_pruned = get_subgraph_by_degree(G, 200)

In [31]:
print(len(G.nodes))
print(len(G.edges))

print(len(G_pruned.nodes))
print(len(G_pruned.edges))

2070
43246
200
8937


In [32]:
G = G_pruned

In [36]:
for u, v in G.edges():
    print(G[u][v]['publication_dates'])

[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 12, 3, 0, 0)]
[datetime.datetime(2020, 

In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

/home/tvg/miniconda3/envs/ml-graphs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
institution_names = [G.nodes[node_id]['affiliated_institution'] for node_id in G.nodes()]
institution_embeddings = sentence_model.encode(institution_names, convert_to_tensor=True).to('cuda')

node_features_list = []
for i, node_id in enumerate(G.nodes()):
    node = G.nodes[node_id]
    scaled_citation_count = torch.tensor([node["citation_count"] / max_citations], dtype=torch.float).to('cuda')
    feat = torch.cat((scaled_citation_count, institution_embeddings[i]))
    node_features_list.append(feat)

node_features = torch.stack(node_features_list)

In [10]:
edge_list = list(G.edges())
edge_indices = torch.empty((2, 0), dtype=torch.long)
edge_features = torch.empty((0, sentence_model.get_sentence_embedding_dimension()), dtype=torch.float).to('cuda')

assert edge_list is not None

node_to_idx = {node: i for i, node in enumerate(G.nodes())}
mapped_edges = [[node_to_idx[u], node_to_idx[v]] for u, v in edge_list]
edge_indices = torch.tensor(mapped_edges, dtype=torch.long).t().contiguous().to('cuda')

all_individual_titles = []
title_slices = []
current_idx = 0
for u, v in edge_list:
    titles = G[u][v]['title']
    all_individual_titles.extend(titles)
    title_slices.append((current_idx, current_idx + len(titles)))
    current_idx += len(titles)

batched_title_embeddings = sentence_model.encode(all_individual_titles, convert_to_tensor=True).to('cuda')

edge_features_list = []
for i, (u, v) in enumerate(edge_list):
    start_idx, end_idx = title_slices[i]
    individual_embeddings_for_edge = batched_title_embeddings[start_idx:end_idx]
    averaged_embedding = individual_embeddings_for_edge.mean(dim=0)
    edge_features_list.append(averaged_embedding)

edge_features = torch.stack(edge_features_list)

In [11]:
graph_data = Data(
    x=node_features,
    edge_index=edge_indices,
    edge_attr=edge_features
)

dataset = [graph_data]
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)


In [12]:
# Assuming 'single_batch_dataloader' is the DataLoader object you created

for batch in dataloader:
    print(f"Type of batch object: {type(batch)}")
    print(f"Batch content: {batch}")
    
    print(f"\nNode features (x):")
    print(f"  Shape: {batch.x.shape}")
    print(f"  Device: {batch.x.device}")
    
    print(f"\nEdge indices (edge_index):")
    print(f"  Shape: {batch.edge_index.shape}")
    print(f"  Device: {batch.edge_index.device}")
    
    print(f"\nEdge features (edge_attr):")
    print(f"  Shape: {batch.edge_attr.shape}")
    print(f"  Device: {batch.edge_attr.device}")

    # You can also check other attributes of the Data object
    print(f"\nNumber of nodes: {batch.num_nodes}")
    print(f"Number of edges: {batch.num_edges}")

    # Break after the first (and only) batch if you only want to inspect it
    break

Type of batch object: <class 'abc.DataBatch'>
Batch content: DataBatch(x=[2070, 385], edge_index=[2, 43246], edge_attr=[43246, 384], batch=[2070], ptr=[2])

Node features (x):
  Shape: torch.Size([2070, 385])
  Device: cuda:0

Edge indices (edge_index):
  Shape: torch.Size([2, 43246])
  Device: cuda:0

Edge features (edge_attr):
  Shape: torch.Size([43246, 384])
  Device: cuda:0

Number of nodes: 2070
Number of edges: 43246
